In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models, losses, optimizers, datasets, utils

In [2]:
from functools import reduce
def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.

    Reference: https://mathieularose.com/function-composition-in-python/
    """
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

In [3]:
def vggBlock(filters,kernel_sizes,name=None):
    if not isinstance(filters, list):
        filters = [filters] * len(kernel_sizes)
    conv_layers = [
        layers.Conv2D(
            i, kernel_size=j, padding='same',
            activation='relu') 
        for i, j in zip(filters, kernel_sizes)]
    
    layers_list = conv_layers + [layers.MaxPooling2D()]
    return compose(*layers_list)

def vggFC(filters, activations="softmax", name=None):
    if not isinstance(filters, list):
        raise ValueError("filters must be a list")
    if activations is None:
        raise ValueError("activations is not null")
    elif not isinstance(activations, list):
        activations = ["relu"] * (len(filters) - 1) + [activations]

    dense_layers = [
        layers.Dense(i, activation=j)
        for i,j in zip(filters, activations)]
    layers_list = [layers.Flatten()] + dense_layers
    return compose(*layers_list)

In [4]:
inp = layers.Input((224, 224, 3))
x = vggBlock(64,[3,3],name="block1")(inp)
x = vggBlock(128,[3,3],name="block2")(x)
x = vggBlock(256,[3,3,3],name="block3")(x)
x = vggBlock(512,[3,3,3],name="block4")(x)
x = vggBlock(512,[3,3,3],name="block5")(x)
out=vggFC([4096,4096,1000],activations="softmax",name="FC")(x)
copy_model=models.Model(inp,out)

2024-04-17 09:32:59.390349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:32:59.390612: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:32:59.396357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:32:59.396613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-17 09:32:59.396808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [5]:
origin_model=tf.keras.applications.VGG16(
    input_shape=(224,224,3),include_top=True)

copy_model.set_weights(origin_model.get_weights())

for org, new in zip(origin_model.layers, copy_model.layers):
    new._name=org.name